## Architektury agentów

### Instalacja bibliotek

In [ ]:
%pip install -U langgraph langchain langchain-openai

### Import bibliotek i konfiguracja

In [2]:
import operator
from typing import Annotated, List, TypedDict, Literal

from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END

from dotenv import load_dotenv
load_dotenv()


True

### Sequential Agent
Sequential Agent prowadzi wewnętrzne rozumowanie w kilku krokach (tzw. scratchpad), ale nie używa narzędzi; stosowany tam, gdzie wystarczy czysta analiza i dedukcja.

In [ ]:

class State(TypedDict):
    question: str
    steps: Annotated[List[str], operator.add]
    answer: str

def plan_node(state: State) -> dict:
    sys = (
        "You are a careful planner. Break the user's question into 2-4 concise steps. "
        "Do not solve. Return only a numbered list of steps; no extra text."
    )
    messages = [("system", sys), ("user", state["question"])]
    resp = llm.invoke(messages)
    raw = resp.content
    steps = []
    for line in str(raw).splitlines():
        line = line.strip()
        if not line:
            continue
        line = line.lstrip("-• ").split(". ", 1)[-1] if ". " in line[:4] else line.lstrip("-• ")
        steps.append(line)
    return {"steps": steps}

def solve_node(state: State) -> dict:
    """Use the planned steps to derive the final answer only."""
    sys = (
        "Use the provided steps to solve the problem. "
        "Return only the final answer, no reasoning."
    )
    messages = [
        ("system", sys),
        ("user", f"Question: {state['question']}\nSteps: {state['steps']}"),
    ]
    resp = llm.invoke(messages)
    return {"answer": str(resp.content).strip()}

# Wire up the graph
graph = StateGraph(State)
graph.add_node("plan", plan_node)
graph.add_node("solve", solve_node)
graph.add_edge(START, "plan")
graph.add_edge("plan", "solve")
graph.add_edge("solve", END)
cot_graph = graph.compile()


In [ ]:
state = {
    "question": "If a book has 350 pages and I read 14 pages per day, how many days to finish?",
    "steps": [],
    "answer": ""
}
out = cot_graph.invoke(state)
print("Final answer:", out["answer"])

### Custom Agent
Custom Agent daje pełną elstyczność, można samemu zdefiniować logikę, routing i węzły.

In [ ]:
class CustomState(TypedDict):
    input: str
    task: Literal["math", "capitalize", "count"]
    result: str

def route(state: CustomState) -> str:
    """Deterministic router based on a simple protocol in the input."""
    text = state["input"].strip().lower()
    if text.startswith("math:"):
        return "math"
    if text.startswith("capitalize:"):
        return "capitalize"
    if text.startswith("count:"):
        return "count"
    return "count"

def do_math(state: CustomState) -> dict:
    expr = state["input"].split(":", 1)[-1].strip()
    allowed = set("0123456789+-*/(). ")
    if any(c not in allowed for c in expr):
        return {"result": "Error: unsupported characters in math expression."}
    try:
        res = eval(expr, {"__builtins__": {}})
    except Exception as e:
        res = f"Error: {e}"
    return {"result": str(res)}

def do_capitalize(state: CustomState) -> dict:
    text = state["input"].split(":", 1)[-1].strip()
    return {"result": text.upper()}

def do_count(state: CustomState) -> dict:
    text = state["input"].split(":", 1)[-1].strip()
    tokens = [t for t in text.split() if t]
    return {"result": f"words={len(tokens)} chars={len(text)}"}

g = StateGraph(CustomState)
g.add_node("math", do_math)
g.add_node("capitalize", do_capitalize)
g.add_node("count", do_count)

# Conditional edges from router
g.add_conditional_edges(
    START,
    route,
    {
        "math": "math",
        "capitalize": "capitalize",
        "count": "count",
    },
)
g.add_edge("math", END)
g.add_edge("capitalize", END)
g.add_edge("count", END)

custom_agent = g.compile()


In [ ]:
# Wypróbuj różne wejścia
for user_input in [
    "math: (12 + 8) * 3",
    "capitalize: langgraph is great!",
    "count: How many words are here?",
]:
    out = custom_agent.invoke({"input": user_input, "task": "count", "result": ""})
    print(f"Input: {user_input}\nResult: {out['result']}\n---")


### Supervisor

In [4]:
from typing import Annotated

class SupervisorState(TypedDict):
    """State for supervisor pattern with multiple agents."""
    topic: str
    messages: Annotated[List[str], operator.add]
    next_agent: str
    final_answer: str


def researcher_agent(state: SupervisorState) -> dict:
    """Researcher agent gathers information about the topic."""
    sys = (
        "You are a researcher. Your job is to gather key facts and information "
        "about the given topic. Provide 2-3 key points. Be concise."
    )
    messages_for_llm = [
        ("system", sys),
        ("user", f"Research this topic: {state['topic']}")
    ]
    resp = llm.invoke(messages_for_llm)
    research_msg = f"RESEARCHER: {resp.content}"
    return {"messages": [research_msg]}


def expert_agent(state: SupervisorState) -> dict:
    """Expert agent analyzes and provides insights based on research."""
    sys = (
        "You are an expert analyst. Review the research provided and give "
        "your expert analysis and conclusions. Be specific and insightful."
    )
    # Get context from previous messages
    context = "\n".join(state["messages"])
    messages_for_llm = [
        ("system", sys),
        ("user", f"Topic: {state['topic']}\n\nPrevious research:\n{context}\n\nProvide your expert analysis.")
    ]
    resp = llm.invoke(messages_for_llm)
    expert_msg = f"EXPERT: {resp.content}"
    return {"messages": [expert_msg]}


def supervisor_agent(state: SupervisorState) -> dict:
    """Supervisor decides which agent should act next or if discussion should end."""
    sys = (
        "You are a supervisor managing a research discussion between a RESEARCHER and an EXPERT. "
        "Based on the conversation so far, decide what should happen next:\n"
        "- Return 'researcher' if we need initial research or more information\n"
        "- Return 'expert' if research is done and we need expert analysis\n"
        "- Return 'end' if both research and expert analysis are complete\n\n"
        "Respond with ONLY one word: researcher, expert, or end"
    )

    context = "\n".join(state["messages"]) if state["messages"] else "No discussion yet"
    messages_for_llm = [
        ("system", sys),
        ("user", f"Topic: {state['topic']}\n\nConversation:\n{context}\n\nWhat's next?")
    ]
    resp = llm.invoke(messages_for_llm)
    next_step = resp.content.strip().lower()

    # Ensure valid response
    if next_step not in ["researcher", "expert", "end"]:
        next_step = "end"

    return {"next_agent": next_step}


def finalize_answer(state: SupervisorState) -> dict:
    """Compile final answer from the discussion."""
    sys = (
        "Summarize the research discussion into a clear, concise final answer. "
        "Include key findings and expert insights."
    )
    context = "\n".join(state["messages"])
    messages_for_llm = [
        ("system", sys),
        ("user", f"Topic: {state['topic']}\n\nDiscussion:\n{context}\n\nProvide final summary:")
    ]
    resp = llm.invoke(messages_for_llm)
    return {"final_answer": resp.content}


def route_supervisor(state: SupervisorState) -> str:
    """Route based on supervisor's decision."""
    next_agent = state.get("next_agent", "researcher")
    if next_agent == "end":
        return "finalize"
    return next_agent

supervisor_graph = StateGraph(SupervisorState)

supervisor_graph.add_node("supervisor", supervisor_agent)
supervisor_graph.add_node("researcher", researcher_agent)
supervisor_graph.add_node("expert", expert_agent)
supervisor_graph.add_node("finalize", finalize_answer)

supervisor_graph.add_edge(START, "supervisor")

supervisor_graph.add_conditional_edges(
    "supervisor",
    route_supervisor,
    {
        "researcher": "researcher",
        "expert": "expert",
        "finalize": "finalize"
    }
)

supervisor_graph.add_edge("researcher", "supervisor")
supervisor_graph.add_edge("expert", "supervisor")

supervisor_graph.add_edge("finalize", END)

supervisor_agent_graph = supervisor_graph.compile()

topic = "What are the main benefits of using LangGraph for building AI agents?"

initial_state = {
    "topic": topic,
    "messages": [],
    "next_agent": "",
    "final_answer": ""
}

result = supervisor_agent_graph.invoke(initial_state)

print(f"TOPIC: {topic}\n")
print("=" * 80)
print("\nDISCUSSION:")
print("-" * 80)
for msg in result["messages"]:
    print(f"\n{msg}\n")
print("=" * 80)
print(f"\nFINAL ANSWER:\n{result['final_answer']}")


TOPIC: What are the main benefits of using LangGraph for building AI agents?


DISCUSSION:
--------------------------------------------------------------------------------

RESEARCHER: 1. **Modular Design**: LangGraph offers a modular architecture that allows developers to easily integrate various components and functionalities, facilitating the rapid development and customization of AI agents.

2. **Enhanced Natural Language Processing**: It leverages advanced natural language processing capabilities, enabling AI agents to understand and generate human-like responses, improving user interaction and engagement.

3. **Scalability and Flexibility**: LangGraph is designed to be scalable, allowing developers to build AI agents that can handle varying workloads and adapt to different use cases, from simple tasks to complex applications.


EXPERT: The research highlights several key benefits of using LangGraph for building AI agents, each of which plays a crucial role in the development and 